<a href="https://colab.research.google.com/github/borricha/-PyTorchZeroToAll/blob/main/lec_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional
import torch.optim as optim
from torchvision import datasets, transforms
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

class NNModel(torch.nn.Module):
    
    def __init__(self):
        super(NNModel,self).__init__()
        self.l1 = nn.Linear(784,520)
        self.l2 = nn.Linear(520,320)
        self.l3 = nn.Linear(320,240)
        self.l4 = nn.Linear(240,120)
        self.l5 = nn.Linear(120,10)
        
    def forward(self,x):
        # input data : ( n , 1 , 28 , 28 )
        x = x.view(-1,784) # Flatten : ( n , 784 )
        x = torch.nn.functional.relu(self.l1(x))
        x = torch.nn.functional.relu(self.l2(x))
        x = torch.nn.functional.relu(self.l3(x))
        x = torch.nn.functional.relu(self.l4(x))
        return self.l5(x)

model = NNModel()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr = 0.01, momentum = 0.5)

model= model.to(device)

def train(epoch):
    model.train()
    for batch_idx,(data,target) in enumerate(train_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)

        optimizer.zero_grad()
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        
        if batch_idx%50==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
            
def test():
    model.eval()
    test_loss=0
    correct=0
    for data,target in test_loader:
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        test_loss += criterion(output,target).item()
        
        pred = output.data.max(1,keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).sum()
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            
            
for epoch in range(1, 9):
    train(epoch)
    test()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.312932
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.304975
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.308303
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.294897
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.297622
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2.294614
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.292645
Train Epoch: 1 [22400/60000 (37%)]	Loss: 2.292964
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.283774
Train Epoch: 1 [28800/60000 (48%)]	Loss: 2.276780
Train Epoch: 1 [32000/60000 (53%)]	Loss: 2.268609
Train Epoch: 1 [35200/60000 (59%)]	Loss: 2.261678
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.236103
Train Epoch: 1 [41600/60000 (69%)]	Loss: 2.225407
Train Epoch: 1 [44800/60000 (75%)]	Loss: 2.166087
Train Epoch: 1 [48000/60000 (80%)]	Loss: 2.067507
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.850344
Train Epoch: 1 [54400/60000 (91%)]	Loss: 1.814040
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.682440

Test set: Average loss: 0.0243, Accuracy: 5370/10000 (54%)